In [3]:
from sklearn.cluster import KMeans
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Define dummy flight plan waypoints (latitude, longitude, altitude)
flight_plan_1 = np.array([[40.7128, -74.0060, 100],
                          [41.8781, -87.6298, 200],
                          [34.0522, -118.2437, 300]])

flight_plan_2 = np.array([[34.0522, -118.2437, 100],
                          [41.8781, -87.6298, 200],
                          [40.7128, -74.0060, 300]])

# Combine all flight plans into a single array
waypoints = np.concatenate((flight_plan_1, flight_plan_2))

# Use K-Means to cluster the waypoints
kmeans = KMeans(n_clusters=2).fit(waypoints)

# Get the cluster centers and counts
cluster_centers = kmeans.cluster_centers_
cluster_counts = np.bincount(kmeans.labels_)

# Identify potential congestion areas
congestion_areas = []
for i in range(len(cluster_centers)):
    if cluster_counts[i] > np.percentile(cluster_counts, 25):
        congestion_areas.append(cluster_centers[i])

# Print the congestion areas
print("Potential congestion areas:")
for area in congestion_areas:
    print(f"Latitude: {area[0]}, Longitude: {area[1]}, Altitude: {area[2]}")

# Function to calculate the distance between two latitudes and longitudes
def distance(lat1, lon1, lat2, lon2):
    R = 6373.0 # approximate radius of Earth in km
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Check for potential conflicts
for i in range(len(flight_plan_1)-1):
    for j in range(len(flight_plan_2)-1):
        d = distance(flight_plan_1[i][0], flight_plan_1[i][1], flight_plan_2[j][0], flight_plan_2[j][1])
        if d < 1:
            print("Potential conflict between UAVs at", flight_plan_1[i], "and", flight_plan_2[j])


C:\Users\vicky\anaconda3\New folder\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\vicky\anaconda3\New folder\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Potential congestion areas:
Latitude: 39.630300000000005, Longitude: -91.877325, Altitude: 250.0
Potential conflict between UAVs at [ 41.8781 -87.6298 200.    ] and [ 41.8781 -87.6298 200.    ]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Define the custom UAV navigation environment
class UAVNavigation:
    def __init__(self):
        self.observation_space = np.zeros(2)
        self.action_space = np.array([0, 1, 2, 3])
        self.goal = np.array([10, 10])
        self.current_pos = np.array([0, 0])
        
    def reset(self):
        self.current_pos = np.array([0, 0])
        return self.current_pos
    
    def step(self, action):
        if action == 0:
            self.current_pos[0] -= 1
        elif action == 1:
            self.current_pos[0] += 1
        elif action == 2:
            self.current_pos[1] -= 1
        elif action == 3:
            self.current_pos[1] += 1
            
        reward = -np.linalg.norm(self.goal - self.current_pos)
        done = np.allclose(self.current_pos, self.goal, atol=1e-1)
        return self.current_pos, reward, done, {}
    
    def render(self):
        pass

# Set up the simulation environment
env = UAVNavigation()

# Define the reinforcement learning algorithm
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=env.observation_space.shape[0]))
model.add(Dense(64, activation='relu'))
model.add(Dense(env.action_space.shape[0], activation='linear'))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam())

# Train the reinforcement learning algorithm
for episode in range(1000):
    state = env.reset()
    for step in range(100):
        action = model.predict(state.reshape(1, -1)).argmax()
        next_state, reward, done, info = env.step(action)
        target = reward + 0.99 * model.predict(next_state.reshape(1, -1)).max()
        target_f = model.predict(state.reshape(1, -1))
        target_f[0][action] = target
        model.fit(state.reshape(1, -1), target_f, epochs=10, verbose=0)
        state = next_state
        if done:
            break

# Test the reinforcement learning algorithm
total_reward = 0
state = env.reset()
for step in range(100):
    env.render()
    action = model.predict(state.reshape(1, -1)).argmax()
    state, reward, done, info = env.step(action)
    total_reward += reward
    if done:
        break
print(f'Total reward: {total_reward}')


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - ETA: 0s